In [28]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

In [29]:
rw = pd.read_csv('data/winequality-red.csv')
rw.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [30]:
rw.quality.value_counts()

quality
5    681
6    638
7    199
4     53
8     18
3     10
Name: count, dtype: int64

In [31]:
rw['target'] = rw.quality.apply(lambda x:1 if x >=6 else 0)
rw.isna().sum().sum()

0

In [32]:
X = rw.iloc[:,:-2].values
y = rw.target.values

In [33]:
X_std = StandardScaler().fit_transform(X)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    X,y,stratify=y,test_size=0.2,random_state=2023
)

In [35]:
from xgboost import XGBClassifier
xgb = XGBClassifier(learning_rate=0.2,max_depth=6)

In [36]:
xgb.fit(X_train,y_train)
pred_xgb =xgb.predict(X_test)
xgb.score(X_test,y_test)

0.840625

In [37]:
params = {
    'max_depth':[5,6,7],
    'learning_rate' : [0.03,0.05,0.07]
}

In [38]:
from sklearn.model_selection import GridSearchCV
grid_xgb = GridSearchCV(xgb,params,scoring='accuracy',cv=5)
grid_xgb.fit(X_test,y_test)
grid_xgb.best_params_

{'learning_rate': 0.05, 'max_depth': 6}

In [51]:
pred_xgb = xgb.predict(X_test)
grid_xgb.best_score_

0.8

In [41]:
lgb = LGBMClassifier()
evals = [(X_test,y_test)]
lgb.fit(X_train,y_train,eval_set=evals,eval_metric='logloss',verbose=True)

[1]	valid_0's binary_logloss: 0.653161
[2]	valid_0's binary_logloss: 0.62463
[3]	valid_0's binary_logloss: 0.595092
[4]	valid_0's binary_logloss: 0.57082
[5]	valid_0's binary_logloss: 0.549098
[6]	valid_0's binary_logloss: 0.531596
[7]	valid_0's binary_logloss: 0.51583
[8]	valid_0's binary_logloss: 0.50344
[9]	valid_0's binary_logloss: 0.492348
[10]	valid_0's binary_logloss: 0.483075
[11]	valid_0's binary_logloss: 0.472153
[12]	valid_0's binary_logloss: 0.464091
[13]	valid_0's binary_logloss: 0.459307
[14]	valid_0's binary_logloss: 0.45308
[15]	valid_0's binary_logloss: 0.447731
[16]	valid_0's binary_logloss: 0.439588
[17]	valid_0's binary_logloss: 0.433313
[18]	valid_0's binary_logloss: 0.43065
[19]	valid_0's binary_logloss: 0.43001
[20]	valid_0's binary_logloss: 0.425872
[21]	valid_0's binary_logloss: 0.421644
[22]	valid_0's binary_logloss: 0.417868
[23]	valid_0's binary_logloss: 0.413767
[24]	valid_0's binary_logloss: 0.411254
[25]	valid_0's binary_logloss: 0.411105
[26]	valid_0's b

c:\Users\YONSAI\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMClassifier()

In [42]:
pred_lgb = lgb.predict(X_test)
lgb.score(X_test,y_test)

0.8375

In [95]:
params = {
    'n_estimators':[100,150,200],
    'max_depth':[4,8,10],
    'min_samples_split':[2,3,4,5,6],
    # 'max_leaf_nodes' : [6,8,10,20]
}

In [99]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2023,n_estimators=130)
rfc.fit(X_train,y_train)
pred_rfc = rfc.predict(X_test)
rfc.score(X_test,y_test)

0.865625

In [96]:
rfc2 = RandomForestClassifier()
grid_sc = GridSearchCV(rfc2,param_grid=params)
grid_sc.fit(X_train, y_train)
grid_sc.best_score_

0.802953431372549